## NOTE : For running this notebook you should download the dataset from :
###    https://www.kaggle.com/datasets/nomihsa965/traffic-signs-dataset-mapillary-and-dfg
##    and then put it in the main directory (Argos/) by "data" naming

In [1]:
import model
!pip3 install -q "flwr[simulation]" "flwr-datasets[vision]" torch torchvision matplotlib

In [2]:
from torch import mps , cuda
import torch


def device_allocation():
    if mps.is_available():
        return torch.device("mps")
    elif cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")




In [3]:
import os

# Server Settings
NUMBER_OF_CLIENTS = 2

# Clients' Settings
CLIENT_BATCH_SIZE = 32
CLIENT_LEARNING_RATE = 0.001
DEVICE = device_allocation()


# Dataset
DATASET_PATH = "../data/"
CLASSES_JSON_FILE = os.path.join(DATASET_PATH, "classes.json")

In [4]:
import torchvision
from torchvision import ops
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch
from tqdm import tqdm


def get_model(num_classes, checkpoint_path=None, device=DEVICE):
    """
    Returns a Faster R-CNN model with the specified number of output classes.

    Args:
        num_classes (int): Number of object classes (including background).
        checkpoint_path (str, optional): Path to load pretrained weights (optional).

    Returns:
        model (torch.nn.Module): Faster R-CNN model ready for training.
    """
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

    # Get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # Replace the pre-trained head with a new one for our custom classes
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


    if checkpoint_path:
        checkpoint = torch.load(checkpoint_path, map_location='cpu')
        model.load_state_dict(checkpoint['model_state_dict'], strict=False)
        print(f"Loaded model weights from: {checkpoint_path}")

    return model.to(device)



def train(model, dataloader, optimizer, device=DEVICE):
    """
    Trains Faster R-CNN model for one epoch.

    Args:
        model (torch.nn.Module): The Faster R-CNN model.
        dataloader (DataLoader): A PyTorch DataLoader returning (images, targets).
        optimizer (torch.optim.Optimizer): Optimizer (e.g., SGD).
        device (torch.device): CUDA or CPU.

    Returns:
        avg_loss (float): Average loss over the epoch.
    """
    model.train()
    model.to(device)
    total_loss = 0.0
    num_batches = len(dataloader)

    pbar = tqdm(dataloader, desc="Training", leave=False)

    for images, targets in pbar:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        total_loss += losses.item()
        pbar.set_postfix(loss=losses.item())

    avg_loss = total_loss / num_batches
    return avg_loss




def evaluate(model, dataloader, device=DEVICE, iou_threshold=0.5):
    """
    Evaluate a Faster R-CNN model on a dataset.

    Args:
        model (torch.nn.Module): Trained Faster R-CNN model.
        dataloader (DataLoader): Validation/test dataloader.
        device (torch.device): "cuda" or "cpu".
        iou_threshold (float): IoU threshold for counting correct detections.

    Returns:
        avg_loss (float): Average loss on dataset.
        accuracy (float): Detection accuracy based on IoU.
    """
    model.eval()
    model.to(device)

    total_loss = 0.0
    num_batches = len(dataloader)
    correct_detections = 0
    total_targets = 0

    with torch.no_grad():
        pbar = tqdm(dataloader, desc="Evaluating", leave=False)

        for images, targets in pbar:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]


            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            total_loss += losses.item()

            outputs = model(images)

            for output, target in zip(outputs, targets):
                pred_boxes = output['boxes']
                pred_labels = output['labels']
                gt_boxes = target['boxes']
                gt_labels = target['labels']

                total_targets += len(gt_boxes)

                if len(pred_boxes) == 0 or len(gt_boxes) == 0:
                    continue

                ious = ops.box_iou(pred_boxes, gt_boxes)

                max_iou_per_gt, matched_preds = ious.max(dim=0)
                correct = (max_iou_per_gt > iou_threshold).sum().item()
                correct_detections += correct

    avg_loss = total_loss / num_batches
    accuracy = correct_detections / total_targets if total_targets > 0 else 0.0

    return avg_loss, accuracy


In [5]:
import json
import logging
import os
from collections import defaultdict
import random
from functools import cache

import torch
from PIL import Image
from torch.utils.data import Subset
from torchvision.transforms import transforms


def extract_label_mapping(classes_file):
    label_map = {}
    try:
        with open(classes_file, 'r') as f:
            json_data = json.load(f)
            for class_name, details in json_data.items():
                if "classIndex" in details:
                    label_map[details["classIndex"]] = class_name
    except FileNotFoundError:
        print(f"Error: Classes file not found at {classes_file}")
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON from {classes_file}. Check file format.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    return label_map


class MTSDDataset(torch.utils.data.Dataset):
    def __init__(
            self,
            root_dir,
            images_dir="images",
            annotations_dir="txts (YOLO)",
            transform=transforms.Compose([transforms.ToTensor()])
    ):

        self.root_directory = root_dir
        self.transform = transform

        self.images_names = sorted(
            os.listdir(
                os.path.join(self.root_directory, images_dir)
            )
        )
        self.full_images_directory = os.path.join(self.root_directory, images_dir)
        self.full_annotations_directory = os.path.join(self.root_directory, annotations_dir)

    def __len__(self):
        return len(self.images_names)

    def __getitem__(self, idx):
        file_name = self.images_names[idx]

        img_path = os.path.join(self.full_images_directory, file_name)
        ann_path = os.path.join(self.full_annotations_directory, file_name[:-4] + '.txt')

        #print(img_path)

        img = Image.open(img_path).convert('RGB')
        w, h = img.size
        img_tensor = self.transform(img)

        objects = []
        try:
            with open(ann_path, 'r') as f:
                for row in f.readlines():
                    objects.append(row.split())
        except FileNotFoundError:
            logging.warning(f"File {ann_path} not found.")
            pass
        # print(objects)

        boxes = []
        labels = []
        for obj in objects:
            # YOLO format: [class_id, x_center, y_center, width, height] (all normalized)
            class_id, cx, cy, bw, bh = map(float, obj)
            xmin = (cx - bw / 2) * w
            ymin = (cy - bh / 2) * h
            xmax = (cx + bw / 2) * w
            ymax = (cy + bh / 2) * h
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(int(class_id))

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        target = {
            'boxes': boxes,
            'labels': labels,
            'image_id': torch.tensor([idx])
        }


        return img_tensor, target


@cache
def partition_dataset(dataset, num_clients):
    label_to_indices = defaultdict(list)

    logging.info(f"Starting Partitioning dataset into {num_clients} clients.")

    for idx in range(len(dataset)):
        _, target = dataset[idx]
        labels = target['labels'].unique().tolist()
        for label in labels:
            label_to_indices[label].append(idx)

    label_ids = list(label_to_indices.keys())
    random.shuffle(label_ids)

    client_data = defaultdict(list)
    for client_id in range(num_clients):
        assigned_labels = label_ids[client_id::num_clients]
        for lbl in assigned_labels:
            client_data[client_id].extend(label_to_indices[lbl])

    logging.info(f" Partitioning dataset into {num_clients} clients finished.")

    return client_data


def get_dataset_for_client(
        partition_id,
        full_dataset,
        partitioned_dataset_indices:dict,
        train_percentage=0.8, val_percentage=0.1, test_percentage=0.1
) -> [Subset , Subset , Subset]:

    assert abs(train_percentage + val_percentage + test_percentage - 1.0) < 1e-4, \
        "Splits must sum to 1.0"

    assert partition_id in list(partitioned_dataset_indices.keys()) , "Client id must be in partitioned dataset keys"

    client_samples = partitioned_dataset_indices[partition_id]
    random.shuffle(client_samples)

    total = len(client_samples)
    train_end = int(total * train_percentage)
    val_end = train_end + int(total * val_percentage)

    train_ids = client_samples[:train_end]
    val_ids = client_samples[train_end:val_end]
    test_ids = client_samples[val_end:]

    train_set = Subset(full_dataset, train_ids)
    val_set = Subset(full_dataset, val_ids)
    test_set = Subset(full_dataset, test_ids)

    return train_set, val_set, test_set



In [6]:
import flwr as fl
import torch.optim as optim

class Client(fl.client.NumPyClient):
    def __init__(
            self,
            model,
            train_dataset,
            eval_dataset,
            device=DEVICE,
            learning_rate=CLIENT_LEARNING_RATE,
            batch_size=CLIENT_BATCH_SIZE,
            optimizer=None,

    ):
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.model = model.to(device)
        self.train_loader = torch.utils.data.DataLoader(
            dataset=train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            collate_fn=lambda x: tuple(zip(*x))
        )
        self.eval_loader = torch.utils.data.DataLoader(
            dataset=eval_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            collate_fn=lambda x: tuple(zip(*x))
        )

        self.device = device
        self.optimizer = optim.SGD(self.model.parameters(), lr=self.learning_rate) if not optimizer else optimizer

    def get_parameters(self, config):
        return [
            val.cpu().numpy() for _, val in self.model.state_dict().items()
        ]

    def set_parameters(self, parameters):
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(v) for k, v in params_dict}
        self.model.load_state_dict(state_dict, strict=True)


    def fit(self, parameters, config):
        self.set_parameters(parameters)
        model.train()
        train(
            model=self.model,
            dataloader=self.train_loader,
            optimizer=self.optimizer,
            device=self.device,
        )

        return self.get_parameters(config={}), len(self.train_loader), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        avg_loss , accuracy = evaluate(
            model=self.model,
            dataloader=self.eval_loader,
        )
        return avg_loss, len(self.eval_loader), {"accuracy": accuracy}



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-31 17:24:38,083	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [7]:
from torch.utils.data import Subset

label_mapping = extract_label_mapping(CLASSES_JSON_FILE)
number_of_classes = len(label_mapping)
dataset = MTSDDataset(root_dir=DATASET_PATH)
dataset = Subset(dataset, list(range(20)))
partitioned_dataset_indices = partition_dataset(dataset=dataset, num_clients=NUMBER_OF_CLIENTS)


In [8]:
partitioned_dataset_indices

defaultdict(list,
            {0: [6,
              13,
              0,
              8,
              15,
              16,
              19,
              5,
              4,
              3,
              8,
              11,
              13,
              14,
              13,
              14,
              10,
              11,
              15,
              14],
             1: [0, 12, 14, 7, 6, 9, 13, 2, 7, 10, 18, 18, 1, 17]})

## Client App

In [9]:
from flwr.common import Context
from flwr.client import ClientApp


def new_client(context : Context) -> Client:
    """Create a Flower client representing a single organization."""

    neural_network = get_model(
        num_classes=number_of_classes
    ).to(DEVICE)

    partition_id = context.node_config["partition-id"]
    train_dataset , val_dataset ,test_dataset = get_dataset_for_client(
        partition_id=partition_id,
        full_dataset=dataset,
        partitioned_dataset_indices=partitioned_dataset_indices,
    )


    return Client(
        model=neural_network,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        learning_rate=CLIENT_LEARNING_RATE,
        batch_size=CLIENT_BATCH_SIZE,
    ).to_client()



client = ClientApp(client_fn=new_client)

## Server App

In [10]:

from flwr.server.strategy import FedAvg

strategy = FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
)

In [11]:
from flwr.common import Context
from flwr.server import ServerAppComponents, ServerConfig, ServerApp


def server_fn(context: Context) -> ServerAppComponents:
    """Construct components that set the ServerApp behaviour.

    You can use the settings in `context.run_config` to parameterize the
    construction of all elements (e.g the strategy or the number of rounds)
    wrapped in the returned ServerAppComponents object.
    """

    # Configure the server for 5 rounds of training
    config = ServerConfig(num_rounds=2,round_timeout=30.0)

    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

In [12]:
# Specify the resources each of your clients need
# By default, each client will be allocated 2x CPU and 0x GPUs

backend_config = {
    "client_resources": {
        "num_cpus": 6, "num_gpus": 0.0
    }
}



In [13]:
from flwr.simulation import run_simulation

run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUMBER_OF_CLIENTS,
    backend_config=backend_config,
)

DEBUG:flwr:Asyncio event loop already running.
INFO :      Starting Flower ServerApp, config: num_rounds=2, round_timeout=30.0s
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
ERROR :     An exception was raised when processing a message by RayBackend
ERROR :     cannot access local variable 'future' where it is not associated with a value
ERROR :     Traceback (most recent call last):
  File "python/ray/_raylet.pyx", line 912, in ray._raylet.prepare_args_internal
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ray/_private/serialization.py", line 519, in serialize
    return self._serialize_to_msgpack(value)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ray/_private/serialization.py", line 497, in _serialize_to_msgpack
    pickle5_serialized_object = self._serialize_to_pickle5(
                                ^^

RuntimeError: Exception in ServerApp thread